In [7]:
import random
import math

In [1]:
def sum_of_squares(v):
    return sum(v_i**2 for v_i in v)

def difference_quotient(f, x, h):
    return (f(x+h) - f(x)) / h

def partial_difference_quotient(f, v, i, h):
    w = [v_j + (h if j == i else 0) for j, v_j in enumerate(v)]
    return (f(w) - f(v)) / h

def estimeate_graient(f, v, h=0.000001):
    return [partial_difference_quotient(f, v, i, h) 
            for i, _ in enumerate(v)]

Using the gradient

In [8]:
def step(v, direction, step_size):
    return [v_i + step_size * direction_i 
            for v_i, direction_i in zip(v, direction)]


def sum_of_squares_gradient(v):
    return [2*vi for vi in v]


#rand point
v = [random.randint(-10, 10) for i in range(3)]
tolerance = 0.00001

while True:
    gradient = sum_of_squares_gradient(v)
    next_v = step(v, gradient, -0.01)
    if math.sqrt(sum((next_vi - vi) ** 2 
                     for next_vi, vi in zip(next_v, v))) < tolerance:
        break
    v = next_v
    
print(v)

[-0.0003618315489830685, -8.040701088512623e-05, 0.00032162804354050493]


Choosing the Right Step Size

In [9]:
step_sizes = [100, 10, 1, 0.1, 0.01, 0.001]
def safe(f):
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')
    return safe_f()

Putting it all together

In [ ]:
def minimize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    """Use gradient descent to find theta that minimize target_fn"""
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001]
    theta= theta_0
    target_fn = safe(target_fn)
    value = target_fn(theta)
    while True:
        gradient = gradient_fn(theta)
        next_thetas = [step(theta, gradient, -step_sizes) 
                       for step_sizes in step_sizes]
        #choose the one that minimize the error function
        next_theta = min(next_thetas, key=target_fn)
        next_value = target_fn(next_theta)
        if abs(value - next_value) < tolerance:
            return theta
        else:
            theta, value = next_thetas, next_value

In [ ]:
def in_random_order(data):
    """generator that returns the elemnts of data in random order"""
    indexes = [i for i, _ in enumerate(data)]
    random.shuffle(indexes)
    for i in indexes:
        yield data[i]
    
        
def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):
    data = zip(x, y)
    theta = theta_0
    #initial guasses
    alpha = alpha_0 #initial step size
    min_theta, min_value = None, float('inf') #the minimum so far
    iterations_with_no_improvement = 0
    #if we ever go 100 iterations with no impovement, stop
    while iterations_with_no_improvement < 100:
        value = sum(target_fn(x_i, y_i, theta) for x_i, y_i, in data)
        if value < min_value:
            #if we've found a new minimum, remeber 
            # it and go back to the original step size 
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
        else:
            #otherwise we're not improving,so try shrinking the step size 
            iterations_with_no_improvement += 1
            alpha *= 0.9
            #and take a gradient step for each of the data points
            for xi, yi in in_random_order(data):
                gradient_i = gradient_fn(xi, yi, theta)
                theta = [ti - alpha * gi 
                         for ti, gi in zip(theta, gradient_i)]
    return min_theta
            